In [1]:
# !export NCCL_IB_DISABLE=1
# !export  CUDA_LAUNCH_BLOCKING=1

In [2]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
# from ipywidgets import FloatProgress
import csv
from optuna import Trial
from typing import Dict, Union, Any
import os
import sys
# notebook_login()

csv.field_size_limit(500 * 1024 * 1024)
CUDA_LAUNCH_BLOCKING=1
# get pwd
notebook_path = os.path.abspath('')

# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'CommitFit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'commitFit' not found in notebook path.")

# Add commitFit directory to Python path, so we can import moudule from commitfit folder directly
if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

from commitfit import CommitFitModel, CommitFitTrainer 


CommitFit


In [3]:
train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

In [4]:
# df

In [5]:
# df = df.rename(columns={'message':'text'})
# train, test = train_test_split(df,test_size=0.996,stratify=df['label'],random_state=1)

In [6]:
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [7]:
# train = df.rename(columns={'3_labels':'label','comment':'text'})

In [8]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)


In [9]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['negative','positive'], sample_size=8)

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [10]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 7094
})

In [11]:
len(train)

7078

In [12]:
# encoded_train = tokenizer(train_dataset['text'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_train["input_ids"].shape)
# encoded_test = tokenizer(test['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
# print(encoded_test["input_ids"].shape)
# # encoded_val = tokenizer(val['comment'].astype(str).to_list(), return_tensors='pt',truncation=True, padding='max_length')

In [13]:
# encoded_train

In [14]:
train['label'].value_counts()

label
negative    4443
positive    2635
Name: count, dtype: int64

In [15]:
test['label'].value_counts()

label
negative    1904
positive    1130
Name: count, dtype: int64

In [16]:
# huggingface-cli login
# train_dataset = CommitDataset(encoded_train, list(train['3_labels']))
# test_dataset = CommitDataset(encoded_test, list(test['3_labels']))
# val_dataset = CommitDataset(encoded_val, list(val['label']))

In [17]:
len(train_dataset)

7094

In [18]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}

In [19]:
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 7094
})

In [20]:
model_id = r"../../sentence-transformers/all-roberta-large-v1"

In [21]:
def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 1e-3, log=True),
        # "num_epochs": trial.suggest_int("num_epochs", 1, 3),
        # "batch_size": trial.suggest_categorical("batch_size", [8, 12, 16]),
        # "seed": trial.suggest_int("seed", 1, 40),
        # "num_iterations": trial.suggest_int("num_iterations", 10, 20),
        # "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }
def model_init(params: Dict[str, Any]) -> CommitFitModel:
    params = params or {}
    # learning_rate = params.get("learning_rate")
    # num_iterations = params.get("num_iterations", 20)
    # solver = params.get("solver", "liblinear")
    # params = {
    #     "head_params": {
    #         # "max_iter": num_iterations,
    #         # "solver": solver,
    #     }
    # }
    return CommitFitModel.from_pretrained(model_id, **params)
def my_compute_objective(metrics):
    print('+++++++++++',metrics)
    return  metrics['accuracy']

In [ ]:
trainer = CommitFitTrainer(
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    model_init=model_init,
    metric = compute_metrics,
    num_iterations=5,
    num_epochs=1
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
[I 2024-05-31 08:14:02,257] A new study created in memory with name: no-name-78f8dc03-528c-4805-bad2-31a752453646
Trial: {'learning_rate': 2.7117920562973035e-06}
model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 70940
  Num epochs = 1
  Total optimization steps = 4434
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4434 [00:00<?, ?it/s]

...

model_head.pkl not found in /CommitFit/sentence-transformers/all-roberta-large-v1, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Generating Training Pairs:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 70940
  Num epochs = 1
  Total optimization steps = 4434
  Total train batch size = 16


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/4434 [00:00<?, ?it/s]

In [ ]:
best_run

In [ ]:
best_run.hyperparameters

In [ ]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

In [ ]:
fewshot_metrics = trainer.evaluate()
fewshot_metrics

In [ ]:
# train.to_csv('train.csv', index=False,encoding = 'utf_8_sig')

In [ ]:
# test.to_csv('test.csv',index=False,encoding = 'utf_8_sig')